### IMPORTS

In [2]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [3]:
from Carga_Descarga import *
from CRM_Central import *

### QUERIES

In [4]:
q = '''WITH barter_table AS (
    SELECT bt.trade.order_id AS order_id,
           bc.campaign_title AS campaign,
           bc.discount.type AS discount_type,
           bc.discount.amount AS discount_amount,
           bc.discount.max_amount AS discount_max_amount,
           bc.rules.min_amount AS min_amount,
           bt.trade.amount_used AS amount_used           
    FROM `peya-data-origins-pro.cl_barter.trades` AS bt 
    LEFT JOIN `peya-data-origins-pro.cl_barter.creates` AS bc ON bt.id = bc.id
    WHERE DATE(bc.created_at) BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -4 MONTH),MONTH) AND CURRENT_DATE()
          AND DATE(bt.tradedAt) BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -4 MONTH),MONTH) AND CURRENT_DATE()
          AND bt.trade.type = 'CONSUME'),
    orders_table AS (
    SELECT o.user.id AS user,
           cn.country_name AS country,
           IFNULL(c.city_name,'-') AS city,
           o.order_status AS status,
           o.order_id AS order_id,
           o.registered_date AS fecha,
           CASE WHEN bt.campaign IS NULL THEN 0 ELSE 1 END AS crm,
           CASE WHEN o.with_logistics THEN o.commission_amount + o.shipping_amount ELSE o.commission_amount END AS revenue,
           IFNULL(o.shipping_amount_no_discount + o.amount_no_discount,0) AS total_paid,
           IFNULL(o.shipping_amount_no_discount + o.amount_no_discount,0) / ce.rate_us AS total_paid_usd,
           IFNULL(bt.amount_used,0) AS amount_used,
           IFNULL(bt.amount_used,0) / ce.rate_us AS amount_used_usd,
           SUM(od.discount_amount) AS total_disc,
           SUM(od.discount_amount) / ce.rate_us AS total_disc_usd,
           SUM(CASE WHEN UPPER(od.discount_type_name) = 'VOUCHER' THEN od.discount_amount ELSE 0 END) AS total_voucher_disc,
           SUM(CASE WHEN UPPER(od.discount_type_name) = 'VOUCHER' THEN od.discount_amount ELSE 0 END) / ce.rate_us AS total_voucher_disc_usd
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o,
    UNNEST(discounts) AS od
    LEFT JOIN barter_table AS bt ON o.order_id = bt.order_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,MONTH) = ce.currency_exchange_date
    WHERE o.registered_date BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -3 MONTH),MONTH) AND CURRENT_DATE()
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,ce.rate_us),
    acq_table AS (
    SELECT ot.user AS user,
           ot.country AS country,
           ot.city AS city,
           ot.fecha AS fecha,
           bt.campaign AS campaign,
           bt.discount_type AS discount_type,
           bt.discount_amount AS discount_amount,
           bt.discount_max_amount AS discount_max_amount,
           bt.min_amount AS min_amount,
           SUM(bt.amount_used) AS amount_used
    FROM orders_table AS ot
    LEFT JOIN barter_table AS bt ON ot.order_id = bt.order_id
    WHERE bt.campaign IS NOT NULL
    GROUP BY 1,2,3,4,5,6,7,8,9),
    cohorts_table AS (
    SELECT acq.user AS user,
           acq.country AS country,
           acq.campaign AS campaign,
           acq.city AS city,
           acq.discount_type AS discount_type,
           acq.discount_amount AS discount_amount,
           acq.discount_max_amount AS discount_max_amount,
           acq.min_amount AS min_amount,
           acq.fecha AS fecha,
           acq.amount_used AS original_amount_used,
           CASE WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) = 0 THEN 'ORIGINALS'
                WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) BETWEEN 1 AND 30 THEN 'M1'
                WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) BETWEEN 31 AND 60 THEN 'M2'
                WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) BETWEEN 61 AND 90 THEN 'M3' END AS cohort,
           IFNULL(COUNT(DISTINCT CASE WHEN ot.status != 'CONFIRMED' THEN NULL ELSE ot.order_id END),0) AS orders,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.revenue END) AS revenue,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_paid END) AS total_paid,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_paid_usd END) AS total_paid_usd,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_disc END) AS total_disc,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_disc_usd END) AS total_disc_usd,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_voucher_disc END) AS total_voucher_disc,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_voucher_disc_usd END) AS total_voucher_disc_usd,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.amount_used END) AS amount_used,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.amount_used_usd END) AS amount_used_usd
    FROM acq_table AS acq
    LEFT JOIN orders_table AS ot ON acq.user = ot.user
    WHERE ot.fecha BETWEEN acq.fecha AND DATE_ADD(acq.fecha, INTERVAL 180 DAY)
    GROUP BY 1,2,3,4,5,6,7,8,9,10,ot.fecha,ot.status)
SELECT FORMAT_DATE('%Y-%m', ct.fecha) AS Month,
       ct.country AS Country,
       ct.city AS City,
       ct.campaign AS Campaign,
       ct.discount_type AS Discount_Type,
       CONCAT('Disc Amount: ',ct.discount_amount,' - Max Amount: ',ct.discount_max_amount,' - MOV: ',ct.min_amount) AS Variation,
       SUM(ct.original_amount_used) AS Original_Amount_Used,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'ORIGINALS' THEN ct.user ELSE NULL END) AS Acquisitions,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'M1' AND ct.orders > 0 THEN ct.user ELSE NULL END) AS M1_Users,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.orders ELSE 0 END) AS M1_Orders,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'M2' AND ct.orders > 0 THEN ct.user ELSE NULL END) AS M2_Users,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.orders ELSE 0 END) AS M2_Orders,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'M3' AND ct.orders > 0 THEN ct.user ELSE NULL END) AS M3_Users,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.orders ELSE 0 END) AS M3_Orders,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.revenue ELSE 0 END) AS M1_Revenue,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_paid ELSE 0 END) AS M1_Total_Paid,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_paid_usd ELSE 0 END) AS M1_Total_Paid_USD,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_disc ELSE 0 END) AS M1_Total_Disc,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_disc_usd ELSE 0 END) AS M1_Total_Disc_USD,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_voucher_disc ELSE 0 END) AS M1_Total_Voucher_Disc,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_voucher_disc_usd ELSE 0 END) AS M1_Total_Voucher_Disc_USD,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.amount_used ELSE 0 END) AS M1_Amount_Used,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.amount_used_usd ELSE 0 END) AS M1_Amount_Used_USD,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.revenue ELSE 0 END) AS M2_Revenue,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_paid ELSE 0 END) AS M2_Total_Paid,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_paid_usd ELSE 0 END) AS M2_Total_Paid_USD,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_disc ELSE 0 END) AS M2_Total_Disc,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_disc_usd ELSE 0 END) AS M2_Total_Disc_USD,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_voucher_disc ELSE 0 END) AS M2_Total_Voucher_Disc,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_voucher_disc_usd ELSE 0 END) AS M2_Total_Voucher_Disc_USD,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.amount_used ELSE 0 END) AS M2_Amount_Used,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.amount_used_usd ELSE 0 END) AS M2_Amount_Used_USD,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.revenue ELSE 0 END) AS M3_Revenue,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_paid ELSE 0 END) AS M3_Total_Paid,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_paid_usd ELSE 0 END) AS M3_Total_Paid_USD,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_disc ELSE 0 END) AS M3_Total_Disc,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_disc_usd ELSE 0 END) AS M3_Total_Disc_USD,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_voucher_disc ELSE 0 END) AS M3_Total_Voucher_Disc,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_voucher_disc_usd ELSE 0 END) AS M3_Total_Voucher_Disc_USD,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.amount_used ELSE 0 END) AS M3_Amount_Used,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.amount_used_usd ELSE 0 END) AS M3_Amount_Used_USD,
FROM cohorts_table AS ct
WHERE UPPER(ct.campaign) NOT LIKE '%%OTHER_CC_%%'
      AND UPPER(ct.campaign) NOT LIKE '%%OTHER_AUTOCOMP_%%'
      AND UPPER(ct.campaign) NOT LIKE '%%OTHER_EMPLOYEE_%%'
      AND UPPER(ct.campaign) NOT LIKE '%%OTHER_OTHER_%%'
      AND UPPER(ct.campaign) NOT LIKE '%%OTHER_NCR_%%'
GROUP BY 1,2,3,4,5,6'''

In [5]:
# Descargo la data
bq_crudo = pd.io.gbq.read_gbq(q, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████| 108102/108102 [01:29<00:00, 1201.22rows/s]


In [6]:
# Copio las bases
crudo = bq_crudo.copy()

### TRABAJO

In [7]:
# Doy formato a las columnas
cols_str = ['Campaign','Country','Month','City','Discount_Type','Variation']
cols_float = [i for i in crudo.columns if i not in cols_str]
crudo[cols_float] = crudo[cols_float].astype(float)
crudo[cols_str] = crudo[cols_str].apply(lambda x: x.astype(str).str.upper())
# Saco los espacios de los nombres de campañas
crudo['Campaign'] = crudo['Campaign'].str.replace(' ', '')
# Doy formato a la fecha
crudo['Month'] = pd.to_datetime(crudo['Month'], format='%Y-%m').dt.strftime('%Y-%m')
# Hago un listado de los paises
countries = list(crudo['Country'].unique())
# Ordeno la base
crudo.sort_values(by=['Country','Month','Campaign','City'],inplace=True)

In [8]:
# Llevo a cabo la limpieza del Reorder
cols_final = ['M1_Users_Final','M2_Users_Final','M3_Users_Final',
              'M1_Orders_Final','M2_Orders_Final','M3_Orders_Final',
              'Acq_M1','Acq_M2','Acq_M3']

for i in cols_final:
    crudo[i] = 0
# Completo las columnas que cree
cols_toFill_m1 = ['M1_Users_Final','M1_Orders_Final','Acq_M1']
cols_fill_m1 = ['M1_Users','M1_Orders','Acquisitions']
cols_toFill_m2 = ['M2_Users_Final','M2_Orders_Final','Acq_M2']
cols_fill_m2 = ['M2_Users','M2_Orders','Acquisitions']
cols_toFill_m3 = ['M3_Users_Final','M3_Orders_Final','Acq_M3']
cols_fill_m3 = ['M3_Users','M3_Orders','Acquisitions']
months = crudo.sort_values(by=['Month'],ascending=False)['Month'].unique().tolist()
if len(months) > 3:
    # Para todos los meses coloco los valores en M1
    crudo[cols_toFill_m1] = crudo[cols_fill_m1].values
    # Para el tercer y cuarto mes coloco los valores en M2
    crudo.loc[crudo['Month'].isin(months[2:]),cols_toFill_m2] = crudo[crudo['Month'].isin(months[2:])][cols_fill_m2].values
    # Para el cuarto mes coloco los valores en M3
    crudo.loc[crudo['Month'] == months[-1],cols_toFill_m3] = crudo[crudo['Month'] == months[-1]][cols_fill_m3].values
else:
    # Para todos los meses coloco los valores en M1
    crudo[cols_toFill_m1] = crudo[cols_fill_m1].values
    # Para el segundo mes coloco los valores en M2
    crudo.loc[crudo['Month'].isin(months[1:]),cols_toFill_m2] = crudo[crudo['Month'].isin(months[1:])][cols_fill_m2].values
    # Para el cuarto mes coloco los valores en M3
    crudo.loc[crudo['Month'] == months[-1],cols_toFill_m3] = crudo[crudo['Month'] == months[-1]][cols_fill_m3].values

In [9]:
# Hago una PT sacando el Month
index = ['Country','City','Campaign','Discount_Type','Variation']
values = list(set(crudo.columns) - set(index + ['Month']))
pt = crudo.pivot_table(index=index,values=values,aggfunc='sum').reset_index()

In [10]:
# Creo las columnas para los segmentos
pt['Benefit'] = pt['Campaign'].apply(lambda x: filtros(x,'BENEFICIOS'))
pt['Type'] = pt['Campaign'].apply(lambda x: filtros(x,'TIPOS'))
pt['Segment'] = pt['Campaign'].apply(lambda x: filtros(x,'SEGMENTOS'))
pt['Level'] = pt['Campaign'].apply(lambda x: filtros(x,'NIVELES'))
pt['Campaña'] = pt['Campaign'].apply(lambda x: filtros(x,'CAMPAIGNS'))
pt['Automated'] = pt['Campaign'].apply(lambda x: filtros(x,'AUTOMATIONS'))
pt['Budget'] = pt['Campaign'].apply(lambda x: filtros(x,'BUDGET'))

In [18]:
pt.to_csv('Stats.csv',index=False)